In [2]:
import instructor
from pydantic import BaseModel
from datetime import datetime, timedelta
from openai import OpenAI
from apscheduler.schedulers.background import BackgroundScheduler
import time
from zoneinfo import ZoneInfo
from tqdm.autonotebook import tqdm
import unicodedata

C:\Users\Admin\AppData\Local\Temp\ipykernel_20540\1655076808.py:8: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### TakeActionIntructor (Function Call with Intructor)

In [9]:
class TakeActionIntructor:
    def __init__(self) -> None:
        llm = OpenAI(
            api_key="lamacpp", 
            base_url="http://zep.hcmute.fit/7500/v1",  
        )
        self.client = instructor.patch(client=llm)
        self.list_keyword = ["appointment", "remind"]
        self.context = {
            "appointment" : " (Thời gian tính theo ngày giờ Việt Nam, timezone Asia/Ho_Chi_Minh)",
            "remind" : " (Thời gian tính theo ngày giờ Việt Nam, timezone Asia/Ho_Chi_Minh)"
        }
        self.action_class = {
            "appointment" : self.AppointmentDetail,
            "remind" : self.RemindDetail
        }
        
    class AppointmentDetail(BaseModel):
        datetime: datetime
        appointment_with: str

    class RemindDetail(BaseModel):
        datetime: datetime 
        remind_description: str

    def check_extract_keyword(self, input_string):
        for keyword in self.list_keyword:
            if input_string.lower().startswith(keyword):
                self.keyword = keyword
                return True, input_string[len(self.keyword):].strip()
        return False, input_string
    
    def take_action(self, input_string):
        check, s = self.check_extract_keyword(input_string)
        if not check:
            print("Keyword not found")
            return None
        
        action = self.client.chat.completions.create(
            model="gemma-2-9b-it",
            response_model=self.action_class[self.keyword],
            messages=[
                {"role": "user", "content": s + self.context[self.keyword]},
            ],
        )
        s = unicodedata.normalize('NFC', s.lower())
        if "ngày mai" in s or "tomorrow" in s:
            action.datetime = datetime(datetime.now().year, datetime.now().month, datetime.now().day, 
                                                    action.datetime.hour, action.datetime.minute, 
                                                    0, 0, ZoneInfo("Asia/Ho_Chi_Minh")) + timedelta(days=1)
        elif "hôm nay" in s or "today" in s:
            action.datetime = datetime(datetime.now().year, datetime.now().month, datetime.now().day, 
                                                    action.datetime.hour, action.datetime.minute, 
                                                    0, 0, ZoneInfo("Asia/Ho_Chi_Minh"))
        elif((action.datetime.month < datetime.now().month) or 
            (action.datetime.month == datetime.now().month and action.datetime.day < datetime.now().day)):
            action.datetime = action.datetime.replace(year=datetime.now().year + 1)
        elif(action.datetime.month >= datetime.now().month and action.datetime.day >= datetime.now().day):
            action.datetime = action.datetime.replace(year=datetime.now().year)
        
        return action

### Test

In [11]:
take_action = TakeActionIntructor()
ta = take_action.take_action("appointment 15h25 hôm nay hẹn với bác sĩ Khoa nhé")
if isinstance(ta, take_action.AppointmentDetail):
    print("Appointment: ",ta.datetime, ta.appointment_with)
elif isinstance(ta, take_action.RemindDetail):
    print("Remind: ", ta.datetime, ta.remind_description)
else:
    print("Not found")

Appointment:  2024-09-17 15:25:00+07:00 Doctor Khoa


In [45]:
take_action = TakeActionIntructor()
#take_action.take_action("remind 10h 15/10/2024 chúng ta tiếp tục tư vấn nhé")
count_true = 0
for i in tqdm(range(100)):
    detail = take_action.take_action("remind 10h 17/01 chúng ta tiếp tục tư vấn nhé")
    datetime_true = datetime(2025, 1, 17, 10, 0)
    if detail.datetime.year == datetime_true.year and detail.datetime.month == datetime_true.month and detail.datetime.day == datetime_true.day and detail.datetime.hour == datetime_true.hour and detail.datetime.minute == datetime_true.minute:
        count_true += 1
print(f"Số lần trùng khớp: {count_true}")

100%|██████████| 100/100 [15:35<00:00,  9.36s/it]

Số lần trùng khớp: 100


In [17]:
take_action = TakeActionIntructor()
#take_action.take_action("remind 10h 15/10/2024 chúng ta tiếp tục tư vấn nhé")
count_true = 0
for i in tqdm(range(100)):
    detail = take_action.take_action("appointment Đặt lịch hẹn với bác sĩ Mai Anh Khoa vào lúc 10h ngày 17/09/2024 nhé")
    datetime_true = datetime(2024, 9, 17, 10, 0)
    if detail.datetime.year == datetime_true.year and detail.datetime.month == datetime_true.month and detail.datetime.day == datetime_true.day and detail.datetime.hour == datetime_true.hour and detail.datetime.minute == datetime_true.minute:
        count_true += 1
print(f"Số lần trùng khớp: {count_true}")

100%|██████████| 100/100 [22:12<00:00, 13.32s/it]

Số lần trùng khớp: 100


### Example With Schedulers

In [8]:
# Create a scheduler
scheduler = BackgroundScheduler()

# Function to execute
def scheduled_appointment(*args):
    print(f"Bạn có lịch hẹn với {args[0]}")

def scheduled_remind(*args):
    print(f"Nhắc nhở bạn đã tới giờ {args[0]}")

scheduler.start()
# Keep the program running
while True:
    check = input()
    if check == "exit":
        scheduler.shutdown()
        break
    elif check == "jobs":
        jobs = scheduler.get_jobs()
        for job in jobs:
            print(f"Job ID: {job.id}, Next Run Time: {job.next_run_time}")
    elif check == "remove":
        jobs = scheduler.get_jobs()
        input_id = input(f"Remove job: ")
        scheduler.remove_job(input_id)
    elif check == "add":
        scheduler.pause()
        query = input("Input query: ")
        take_action = TakeActionIntructor()
        ta = take_action.take_action(query)
        if isinstance(ta, take_action.AppointmentDetail):
            print("Appointment: ",ta.datetime, ta.appointment_with)
            ta.datetime= ta.datetime.replace(tzinfo=ZoneInfo("Asia/Ho_Chi_Minh"))
            scheduler.add_job(scheduled_appointment, 'date', run_date=ta.datetime, args=[ta.appointment_with])
        elif isinstance(ta, take_action.RemindDetail):
            print("Remind: ", ta.datetime, ta.remind_description)
            ta.datetime= ta.datetime.replace(tzinfo=ZoneInfo("Asia/Ho_Chi_Minh"))
            scheduler.add_job(scheduled_remind, 'date', run_date=ta.datetime, args=[ta.remind_description])
        scheduler.resume()

    time.sleep(1)


Remind:  2024-09-17 17:31:00+07:00 Tiếp tục tư vấn
Job ID: 06b7b9043c2e49ea8da97b0031ba3b9b, Next Run Time: 2024-09-17 17:31:00+07:00
Remind:  2024-09-17 17:33:00+07:00 Tiếp tục tư vấn
Job ID: 06b7b9043c2e49ea8da97b0031ba3b9b, Next Run Time: 2024-09-17 17:31:00+07:00
Job ID: 654aa84005634d5f8283e672711b0177, Next Run Time: 2024-09-17 17:33:00+07:00
Nhắc nhở bạn đã tới giờ Tiếp tục tư vấn
Nhắc nhở bạn đã tới giờ Tiếp tục tư vấn


### Summary of Questions

In [88]:
import enum
class QuestionType(str, enum.Enum):
    """Enumeration for single-label text classification."""
    issues = "Vấn đề phụ nữ sau sinh (tâm lý, sinh lý, gia đình, ...)"
    orther = "Vấn đề khác"
    
class QuestionInstructor:
    def __init__(self) -> None:
        llm = OpenAI(
            api_key="lamacpp", 
            base_url="http://zep.hcmute.fit/7500/v1",  
        )
        self.client = instructor.patch(client=llm)

    class SummaryOfQuestion(BaseModel):
        question: str
        question_type: QuestionType

    def take_question(self, input_string):
        action = self.client.chat.completions.create(
            model="gemma-2-9b-it",
            response_model=self.SummaryOfQuestion,
            messages=[
                {"role": "user", "content": f"Tóm tắt câu hỏi bên dưới để phục vụ cho tra cứu google và kiểm tra câu hỏi đó có phải là vấn đề của phụ nữ sau sinh (tâm lý, sinh lý, gia đình, ...) hay không : \
                                    \nCâu hỏi: {input_string}\
                                    \n(Sử dụng ngôn ngữ tiếng Việt)"},
            ],
        )
        return action

In [87]:
data = "Vợ tôi mới sinh con được hơn 1 tháng, gần đây cô ấy bắt đầu có những biển hiện rất lạ như ít nói, không muốn tiếp xúc với mọi người, mặc dù trước đây rất hay trò chuyện vui vẻ với mọi người. Không những vậy cô ấy còn hay cáu kỉnh, dễ tức giận, và rất hay buồn ngủ vào ban ngày. Như vậy có phải vợ tôi bị trầm cảm sau sinh không? Tôi phải làm như thế nào?"

action = QuestionInstructor().take_question(data)
print(action)

question='Vợ (sau sinh 1 tháng) thay đổi tính cách: ít nói,  khép kín, cáu kỉnh, dễ buồn ngủ. Có bị trầm cảm sau sinh không?' question_type=<QuestionType.issues: 'Vấn đề phụ nữ sau sinh (tâm lý, sinh lý, gia đình, ...)'>


#### Test

In [82]:
import random

# Tạo dữ liệu mẫu cho nhãn 'issues'
issues_samples = [
    "Tôi cảm thấy lo lắng sau khi sinh con, không biết phải làm sao.",
    "Mối quan hệ với chồng tôi trở nên khó khăn sau khi sinh con.",
    "Tôi bị mất ngủ sau khi sinh và điều này ảnh hưởng rất nhiều đến tâm lý của tôi.",
    "Cảm giác mệt mỏi, đau nhức sau khi sinh là điều tôi không ngờ tới.",
    "Việc chăm sóc con và gia đình sau khi sinh làm tôi cảm thấy bị căng thẳng."
]

# Tạo dữ liệu mẫu cho nhãn 'other'
other_samples = [
    "Tôi thích đi du lịch và khám phá những nơi mới.",
    "Thời tiết hôm nay thật đẹp, rất thích hợp để đi dạo.",
    "Tôi vừa hoàn thành một cuốn sách rất hay về phát triển bản thân.",
    "Năm nay tôi có kế hoạch học thêm một ngoại ngữ mới.",
    "Tôi thích xem phim vào cuối tuần để thư giãn."
]

# Sinh dữ liệu ngẫu nhiên
def generate_classification_data(num_samples=10):
    data = []
    for _ in range(num_samples):
        if random.random() > 0.5:
            label = "issues"
            text = random.choice(issues_samples)
            data.append({"question": text, "type": QuestionType.issues})
        else:
            text = random.choice(other_samples)
            data.append({"question": text, "type": QuestionType.orther})
    
    return data

# Sử dụng hàm sinh dữ liệu
sample_data = generate_classification_data(50)

In [83]:
count_true = 0
for question in sample_data:
    action = QuestionInstructor().take_question(question.get("question"))

    if action.question_type == question.get("type"):
        count_true += 1
print(f"Số lần trùng khớp: {count_true}")
    

Số lần trùng khớp: 46


### Emotion Classification

In [13]:
import instructor
from pydantic import BaseModel
from openai import OpenAI
import enum

class EmotionText(str, enum.Enum):
    """Enumeration for single-label text classification."""
    TIEU_CUC = "Cảm xúc tiêu cực"
    TICH_CUC = "Cảm xúc tích cực"

class Emotion_Classification:
    def __init__(self) -> None:
        llm = OpenAI(
            api_key="lamacpp", 
            base_url="http://zep.hcmute.fit/7500/v1",  
        )
        self.client = instructor.patch(client=llm)
    
    class SingleEmotionPrediction(BaseModel):
        """
        Class for a single emotion class label prediction.
        """

        emotion: EmotionText
        
    def classify(self, data: str):
        return self.client.chat.completions.create(
            model="gemma-2-9b-it",
            response_model=self.SingleEmotionPrediction,
            messages=[
                {
                    "role": "user",
                    "content": f"Cảm xúc tích cực hay tiêu cực qua đoạn văn bản sau: {data}",
                },
            ],
        )

In [14]:
prediction = Emotion_Classification().classify("Tôi đang đau đầu, mệt mỏi, căng thẳng quá, hơi quạo")
print(prediction)

emotion=<EmotionText.TIEU_CUC: 'Cảm xúc tiêu cực'>


### Test

In [15]:
count_true = 0
for i in range(100):
    prediction = Emotion_Classification().classify("Tôi cảm thấy khá tuyệt vọng, muốn buôn xuôi")
    if prediction.emotion == EmotionText.TIEU_CUC:
        count_true += 1
print(count_true)

100


In [10]:
count_true = 0
for i in range(100):
    prediction = Emotion_Classification().classify("Tôi cảm thấy tốt hơn rồi, cảm ơn bạn")
    if prediction.emotion == EmotionText.TICH_CUC:
        count_true += 1
print(count_true)

100
